In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# Definimos path
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'


In [3]:
with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerados.pickle', 'rb') as file:
    dict_cod_aglomerado = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia.pickle', 'rb') as file:
    dict_cod_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_provincia.pickle', 'rb') as file:
    map_aglomerado_provincia = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_aglomerado_region.pickle', 'rb') as file:
    map_aglomerado_region = pickle.load(file)

with open(pathdata + 'modulos/diccionarios/' + 'diccionario_provincia_region.pickle', 'rb') as file:
    map_provincia_region = pickle.load(file)

In [4]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [5]:
pondera = 'PONDERA_new'
var = 'PROVINCIA'
df_temp = df_people_pool.query('CH06 >= 14')

In [6]:
df_ocupados = df_temp[df_temp['ESTADO']==1][[var, pondera, 'CH04']]
df_ocupados_publico = df_temp[(df_temp['ESTADO']==1) & (df_temp['PP04A']==1)][[var, pondera, 'CH04']]
df_tasa = df_ocupados_publico.groupby(['CH04',var])[pondera].sum() / df_ocupados.groupby(['CH04',var])[pondera].sum()
df_empleo_publico = df_tasa.unstack(level=0)*100
df_empleo_publico = df_empleo_publico.rename(columns={1:'Varon', 2:'Mujer'})
df_empleo_publico['Ratio'] = df_empleo_publico['Mujer']/df_empleo_publico['Varon']*100
df_empleo_publico.index = df_empleo_publico.index.map(dict_cod_provincia)

size = df_ocupados_publico.groupby(['CH04', var]).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
n_pob = df_ocupados_publico.groupby(['CH04', var])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']
size.index = size.index.map(dict_cod_provincia)
empleo_publico = pd.concat([df_empleo_publico, size], axis=1)
empleo_publico = empleo_publico.sort_index()
empleo_publico.to_csv(pathdata + 'data_output/indicadores_auxiliares/01_empleo_publico.csv')

In [7]:
df_ocupados_intensi = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04','INTENSI']]
df_tasa = df_ocupados_intensi.groupby(['CH04',var,'INTENSI'])[pondera].sum()/df_ocupados_intensi.groupby(['CH04',var])[pondera].sum()
df_intensidad_laboral = df_tasa.unstack(level=0)*100
df_intensidad_laboral = df_intensidad_laboral.rename(columns={1:'Varon', 2:'Mujer'})
df_intensidad_laboral['Ratio'] = df_intensidad_laboral['Mujer']/df_intensidad_laboral['Varon']*100
new_index = df_intensidad_laboral.index.get_level_values('PROVINCIA').map(dict_cod_provincia)
df_intensidad_laboral.index = pd.MultiIndex.from_tuples(zip(new_index, df_intensidad_laboral.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

size = df_ocupados_intensi.groupby(['CH04', var, 'INTENSI']).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
size.index = pd.MultiIndex.from_tuples(zip(new_index, size.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

n_pob = df_ocupados_intensi.groupby(['CH04', var, 'INTENSI'])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
n_pob.index = pd.MultiIndex.from_tuples(zip(new_index, n_pob.index.get_level_values('INTENSI')), names=['PROVINCIA', 'INTENSI'])

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']
intensidad_laboral = pd.concat([df_intensidad_laboral, size], axis=1)
intensidad_laboral = intensidad_laboral.sort_index()

intensidad_laboral.to_csv(pathdata + 'data_output/indicadores_auxiliares/02_intensidad_laboral.csv')

In [8]:
df_ocupados_educacion = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'CH04','CH12','CH13']]
df_ocupados_educacion['Educacion'] = pd.NaT

df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([1,2,3,9])),'Educacion'] = 'Hasta secundario incompleto'
df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([4,5])) & (df_ocupados_educacion['CH13']==2),'Educacion'] = 'Hasta secundario incompleto'
df_ocupados_educacion.loc[(df_ocupados_educacion['CH12'].isin([4,5])) & (df_ocupados_educacion['CH13']==1),'Educacion'] = 'Secundario completo'
df_ocupados_educacion.loc[df_ocupados_educacion['CH12'].isin([6,7,8]),'Educacion'] = 'Educación superior'
df_ocupados_educacion = df_ocupados_educacion[~df_ocupados_educacion['Educacion'].isna()]

df_tasa = df_ocupados_educacion.groupby(['CH04',var,'Educacion'])[pondera].sum()/df_ocupados_educacion.groupby(['CH04',var])[pondera].sum()

df_educacion = df_tasa.unstack(level=0)*100
df_educacion = df_educacion.rename(columns={1:'Varon', 2:'Mujer'})
df_educacion['Ratio'] = df_educacion['Mujer']/df_educacion['Varon']*100

new_index = df_educacion.index.get_level_values('PROVINCIA').map(dict_cod_provincia)
df_educacion.index = pd.MultiIndex.from_tuples(zip(new_index, df_educacion.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

size = df_ocupados_educacion.groupby(['CH04', var, 'Educacion']).size().unstack(level=0)
size = size.rename(columns= {1:'N_v', 2:'N_m'})
size.index = pd.MultiIndex.from_tuples(zip(new_index, size.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

n_pob = df_ocupados_educacion.groupby(['CH04', var, 'Educacion'])[pondera].sum().unstack(level=0)
n_pob = n_pob.rename(columns= {1:'N_pob_v', 2:'N_pob_m'})
n_pob.index = pd.MultiIndex.from_tuples(zip(new_index, n_pob.index.get_level_values('Educacion')), names=['PROVINCIA', 'Educacion'])

size[['N_pob_v', 'N_pob_m']] = n_pob[['N_pob_v', 'N_pob_m']]
size['N_pob_tot'] = n_pob['N_pob_v'] + n_pob['N_pob_m']

educacion = pd.concat([df_educacion, size], axis=1)
educacion = educacion.sort_index()

educacion.to_csv(pathdata + 'data_output/indicadores_auxiliares/03_maximo_nivel_educativo.csv')

In [23]:
df_ocupados = df_temp[(df_temp['ESTADO']==1)][[var, pondera, 'P21_new', 'CH04']]
df_ocupados = df_ocupados[df_ocupados['P21_new']>0]

In [24]:
df_ocupados['Multiplication'] = df_ocupados['P21_new'] * df_ocupados[pondera]
numerador = df_ocupados.groupby(['CH04', var])['Multiplication'].sum().unstack(level=0)
denominador = df_ocupados.groupby(['CH04', var])[pondera].sum().unstack(level=0)
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa.columns = ['Varon', 'Mujer']

In [26]:
tasa['Mujer'] / tasa['Varon']

PROVINCIA
2     0.825809
6     0.720321
10    0.830330
14    0.693172
18    0.846427
22    0.874493
26    0.664494
30    0.658217
34    0.874688
38    0.741948
42    0.710414
46    0.882030
50    0.708517
54    0.755043
58    0.741717
62    0.849890
66    0.729896
70    0.779888
74    0.801567
78    0.697654
82    0.730241
86    0.814533
90    0.746476
94    0.896363
dtype: float64

In [27]:
def weighted_median(values, weights):
    sorted_indices = np.argsort(values)
    sorted_values = values[sorted_indices]
    sorted_weights = weights[sorted_indices]
    cumulative_weights = np.cumsum(sorted_weights)
    total_weight = cumulative_weights[-1]
    median_index = np.searchsorted(cumulative_weights, total_weight / 2.0)
    return sorted_values[median_index]

def weighted_median_group(x):
    return weighted_median(x['P21_new'].values, x[pondera].values)


df_mediana = df_ocupados.groupby(['CH04', var]).apply(weighted_median_group)
tasa = df_mediana.unstack(level=0)
tasa.columns = ['Varon', 'Mujer']

In [28]:
tasa['Mujer'] / tasa['Varon']

PROVINCIA
2     0.900000
6     0.682948
10    0.800000
14    0.653679
18    0.813467
22    0.813467
26    0.750000
30    0.625000
34    0.800000
38    0.732120
42    0.709241
46    0.900000
50    0.666667
54    0.666667
58    0.770653
62    0.882353
66    0.658908
70    0.788922
74    0.762625
78    0.727273
82    0.750000
86    0.800000
90    0.585384
94    0.833333
dtype: float64

In [4]:
df_people_3T2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', low_memory=False, index_col=0, delimiter=';')

In [5]:
df_temp = df_people_3T2021.query('CH06 >= 14')

df_estado = df_temp[(df_temp['ESTADO']==1)]          # personas ocupadas
df_ocupados = df_temp[(df_temp['P21']!=-9) & (df_temp['ESTADO']==1) & (df_temp['CAT_OCUP']==3)][['PONDIIO', 'P21', 'CH04']]
df_ocupados = df_ocupados[df_ocupados['P21']>0]


In [6]:
df_ocupados['Multiplication'] = df_ocupados['P21'] * df_ocupados['PONDIIO']
numerador = df_ocupados.groupby(['CH04'])['Multiplication'].sum()
denominador = df_ocupados.groupby(['CH04'])['PONDIIO'].sum()
tasa = numerador.div(denominador, fill_value=np.nan)    
tasa = tasa.rename(index= {1:'Varon', 2:'Mujer'})
tasa.loc['Ratio'] = tasa['Mujer']/tasa['Varon']*100
tasa

CH04
Varon    54308.106493
Mujer    40695.930279
Ratio       74.935277
dtype: float64